In [ ]:
import pandas as pd
import numpy as np
import torch
from umap import UMAP
import plotly_express as px
import plotly.graph_objects as go

In [ ]:
def plot_summarize(dfs, columns, meta_batch_size):
    dfs[0] = dfs[0].groupby(np.arange(len(dfs[0])) // meta_batch_size).mean()
    dfs[0].reset_index(drop=True, inplace=True)
    dfs[1].reset_index(drop=True, inplace=True)
    df = pd.concat([dfs[0][columns], dfs[1][columns]], axis=1)
    df.columns = [str(c)+'_train' for c in columns] + \
        [str(c)+'_valid' for c in columns]
    df[['accuracy_train', 'accuracy_valid']] = 100 * \
        df[['accuracy_train', 'accuracy_valid']]
    df['iter'] = df.index

    dfs[2].columns = list(dfs[2].columns[1:]) + ['model_name']
    test = dfs[2].groupby('model_name', as_index=False)[dfs[2].columns].mean()
    test['iter'] = test['model_name'].apply(
        lambda x: int(x[x.index('_') + 1:-3]))
    test['accuracy'] = 100 * test['accuracy']
    test.drop(columns=['model_name'], inplace=True)
    test.sort_values(by=['iter'], inplace=True)

    # Summarizing test scores
    ind = test[test['accuracy'] == test['accuracy'].max()]['iter'].values[0]
    t = dfs[2][dfs[2]['model_name'] == 'model_' + str(ind) + '.pt']
    t['accuracy'] = 100 * t['accuracy']
    print('Test Metrics Stats of Best Model at {}-th Iteration'.format(ind))
    display(t.describe())

    # Plotting
    ctv = [str(c)+'_train' for c in columns] + \
        [str(c)+'_valid' for c in columns]
    fig = px.line(df, x=df.index, y=ctv)
    # figure this test.columns format and how to name
    fig.add_scatter(x=test['iter'], y=test['accuracy'], name='accuracy_test')
    fig.show()

    return df, test

In [ ]:
def viz_images(experiment, epoch, mode):
    images = torch.load(
        '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/logs/DELPO_{}_5-way_5-shot_15-queries/{}/images_epoch-{}_{}.pt'.format(experiment[1], experiment[0], epoch, mode))
    
    fig1 = px.imshow(np.array(images['gt_examples'].permute(0,2,3,1))[::15], facet_col=0 ,facet_col_wrap=5)
    fig2 = px.imshow(np.array(images['reconst_examples'].permute(0,2,3,1))[::15], facet_col=0 ,facet_col_wrap=5)

    fig1.show()
    fig2.show()

In [ ]:
def viz_latents(experiment, epoch, mode):

    #Only visualizing the image-latent-gaussian distribution mu's
    latents = torch.load(
        '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/logs/DELPO_{}_5-way_5-shot_15-queries/{}/latents_epoch-{}_{}.pt'.format(experiment[1], experiment[0], epoch, mode))
    df = pd.DataFrame(np.array(latents['label_latents'][0]))
    df['class'] = pd.Series(list(np.full((15,), 0)) + list(np.full((15,), 1)) + list(
        np.full((15,), 2)) + list(np.full((15,), 3)) + list(np.full((15,), 4)))
    features = df.iloc[:, :-1]

    umap = UMAP(n_components=3, init='random', random_state=0)

    proj_2d = umap.fit_transform(features)

    fig_3d = px.scatter_3d(
        proj_2d, x=0, y=1, z=2,
        color=df['class'], labels={'color': 'class'}
    )
    fig_3d.show()


# MiniImagenet: 5-way, 5-shot 15-queries 

# Experiment-1:

In [ ]:
df_train = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp1/train.csv')
df_valid = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp1/valid.csv')
df_test = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp1/test.csv')

In [ ]:
plot_summarize([df_train, df_valid, df_test], ['accuracy', 'ELBO', 'Label_KL', 'Style_KL', 'Reconst_Loss', 'CE_Loss'], 10)

# Experiment-3:

In [ ]:
df_train = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp3/train.csv')
df_valid = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp3/valid.csv')
df_test = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp3/test.csv')

In [ ]:
df = plot_summarize([df_train, df_valid, df_test], ['accuracy', 'ELBO', 'Label_KL', 'Style_KL', 'Reconst_Loss', 'CE_Loss'], 10)

# Experiment-4:

In [ ]:
df_train = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp4/train.csv')
df_valid = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp4/valid.csv')
df_test = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp4/test.csv')

In [ ]:
df = plot_summarize([df_train, df_valid, df_test], ['accuracy', 'ELBO', 'Label_KL', 'Style_KL', 'Reconst_Loss', 'CE_Loss'], 30)

# Experiment-5:

In [ ]:
df_train = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp5/train.csv')
df_valid = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp5/valid.csv')
df_test = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp5/test.csv')

In [ ]:
acc = 100 * np.array(df_test.accuracy)
1.96 * acc.std()/(600**0.5)

In [ ]:
df = plot_summarize([df_train, df_valid, df_test], ['accuracy', 'ELBO', 'Label_KL', 'Style_KL', 'Reconst_Loss', 'CE_Loss'], 20)

In [ ]:
viz_images('exp5', 6000, 'train')

In [ ]:
viz_latents('exp5', 6000, 'train')

# Experiment-3 (W/O Reparametrization):

In [ ]:
df_train = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp3r/train.csv')
df_valid = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp3r/valid.csv')
df_test = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp3r/test.csv')

In [ ]:
df = plot_summarize([df_train, df_valid, df_test], ['accuracy', 'ELBO', 'Label_KL', 'Style_KL', 'Reconst_Loss', 'CE_Loss'], 10)

# Experiment-7: 

In [ ]:
df_train = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp7/train.csv')
df_valid = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp7/valid.csv')
df_test = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp7/test.csv')

In [ ]:
acc = 100 * np.array(df_test.accuracy)
1.96 * acc.std()/(600**0.5)

In [ ]:
df = plot_summarize([df_train, df_valid, df_test], ['accuracy', 'ELBO', 'Label_KL', 'Style_KL', 'Reconst_Loss', 'CE_Loss'], 20)

In [ ]:
viz_images('exp7', 6000, 'train')

In [ ]:
viz_latents('exp7', 13000, 'train')

# Experiment-8:

In [ ]:
df_train = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp8/train.csv')
df_valid = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp8/valid.csv')
df_test = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp8/test_all.csv')

df_test = df_test[~(df_test['CE_Loss'] == 'model_last.pt')]

df_train1 = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp8/train_45.csv')
df_valid1 = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp8/valid_45.csv')
df_test1 = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp8/test_all_45.csv')

df_test1 = df_test1[~(df_test1['CE_Loss'] == 'model_last.pt')]

df_train_all = pd.concat([df_train, df_train1])
df_valid_all = pd.concat([df_valid, df_valid1])
df_test_all = pd.concat([df_test, df_test1])

In [ ]:
df, test = plot_summarize(
    [df_train_all, df_valid_all, df_test_all],
    ['accuracy', 'ELBO', 'Label_KL', 'Style_KL', 'Reconst_Loss', 'CE_Loss'],
    20)

In [ ]:
viz_images(['exp8', 'miniimagenet'], 41500, 'train')

In [ ]:
viz_latents(['exp8', 'miniimagenet'], 41500, 'train')

# Experiment-8-arch-ablation: 

added a layer before extracting mu, sigma for latent distribution, thus now: 1000 -- 256 -- 64 

In [ ]:
df_train = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp8-archab/train.csv')
df_valid = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp8-archab/valid.csv')
df_test = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp8-archab/test_all.csv')
df_test = df_test[~(df_test['CE_Loss'] == 'model_last.pt')]

In [ ]:
df, test = plot_summarize(
    [df_train, df_valid, df_test],
    ['accuracy', 'ELBO', 'Label_KL', 'Style_KL', 'Reconst_Loss', 'CE_Loss'],
    20)

# Experiment-8-TAD: 

## EAEN 

In [ ]:
df_train = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_10-queries/exp8-eaen/train.csv')
df_valid = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_10-queries/exp8-eaen/valid.csv')
df_test = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_10-queries/exp8-eaen/test_all.csv')
df_test = df_test[~(df_test['CE_Loss'] == 'model_last.pt')]

df_train1 = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_10-queries/exp8-eaen/train_33500.csv')
df_valid1 = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_10-queries/exp8-eaen/valid_33500.csv')
df_test1 = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_10-queries/exp8-eaen/test_all_33000.csv')
df_test1 = df_test1[~(df_test1['CE_Loss'] == 'model_last.pt')]

df_train_all = pd.concat([df_train, df_train1])
df_valid_all = pd.concat([df_valid, df_valid1])
df_test_all = pd.concat([df_test, df_test1])

In [ ]:
df, test = plot_summarize(
    [df_train_all, df_valid_all, df_test_all],
    ['accuracy', 'ELBO', 'Label_KL', 'Style_KL', 'Reconst_Loss', 'CE_Loss'],
    20)

# Experiment-8-lr-ablations:

a ; b ; d :: 0.003,0.0003 ; 0.006,0.0006 ; 0.01,0.001   ---- (inner lr, meta-lr) 

In [ ]:
test = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp8c/test.csv')
df_testa = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp8c/exp8ca/test.csv')
df_testb = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp8c/exp8cb/test.csv')
df_testd = pd.read_csv('./logs/DELPO_miniimagenet_5-way_5-shot_15-queries/exp8c/exp8cd/test.csv')

In [ ]:
df_testa.columns = list(df_testa.columns[1:]) + ['model_name']
a = df_testa.groupby('model_name', as_index=False).mean()
a['iter'] = a['model_name'].apply(lambda x: int(x[x.index('_')+1 : -3]))
a

In [ ]:
df_testb.columns = list(df_testb.columns[1:]) + ['model_name']
df_testb.groupby('model_name', as_index=False)['accuracy'].mean()

In [ ]:
df_testd.columns = list(df_testd.columns[1:]) + ['model_name']
df_testd.groupby('model_name', as_index=False)['accuracy'].mean()

In [ ]:
test.columns = list(test.columns[1:]) + ['model_name']
test = test.groupby('model_name', as_index=False).mean()
test['iter'] = test['model_name'].apply(lambda x: int(x[x.index('_')+1 : -3]))
test.drop(columns=['model_name'], inplace=True)
test

# Omniglot: 5-way, 5-shot, 15-queries

# Experiment-1: 

In [ ]:
df_train = pd.read_csv('./logs/DELPO_omniglot_5-way_5-shot_15-queries/exp1/train.csv')
df_valid = pd.read_csv('./logs/DELPO_omniglot_5-way_5-shot_15-queries/exp1/valid.csv')
df_test = pd.read_csv('./logs/DELPO_omniglot_5-way_5-shot_15-queries/exp1/test.csv')

In [ ]:
acc = 100 * np.array(df_test.accuracy)
1.96 * acc.std()/(600**0.5)

In [ ]:
df = plot_summarize([df_train, df_valid, df_test], ['accuracy', 'ELBO', 'Label_KL', 'Style_KL', 'Reconst_Loss', 'CE_Loss'], 20)

In [ ]:
viz_images(['exp4', 'omniglot'], 5000, 'train')

In [ ]:
viz_latents('exp7', 13000, 'train')

# Experiment-2: 

In [ ]:
df_train = pd.read_csv('./logs/DELPO_omniglot_5-way_5-shot_15-queries/exp2/train.csv')
df_valid = pd.read_csv('./logs/DELPO_omniglot_5-way_5-shot_15-queries/exp2/valid.csv')
df_test = pd.read_csv('./logs/DELPO_omniglot_5-way_5-shot_15-queries/exp2/test.csv')

In [ ]:
df_train

In [ ]:
acc = 100 * np.array(df_test.accuracy)
1.96 * acc.std()/(600**0.5)

In [ ]:
df = plot_summarize([df_train, df_valid, df_test], ['accuracy', 'ELBO', 'Label_KL', 'Style_KL', 'Reconst_Loss', 'CE_Loss'], 20)

In [ ]:
viz_images(['exp4', 'omniglot'], 5000, 'train')

In [ ]:
viz_latents('exp7', 13000, 'train')

# Experiment-3: 

In [ ]:
df_train = pd.read_csv('./logs/DELPO_omniglot_5-way_5-shot_15-queries/exp3/train.csv')
df_valid = pd.read_csv('./logs/DELPO_omniglot_5-way_5-shot_15-queries/exp3/valid.csv')
df_test = pd.read_csv('./logs/DELPO_omniglot_5-way_5-shot_15-queries/exp3/test.csv')

In [ ]:
df_train

In [ ]:
acc = 100 * np.array(df_test.accuracy)
1.96 * acc.std()/(600**0.5)

In [ ]:
df = plot_summarize([df_train, df_valid, df_test], ['accuracy', 'ELBO', 'Label_KL', 'Style_KL', 'Reconst_Loss', 'CE_Loss'], 20)

In [ ]:
viz_images(['exp4', 'omniglot'], 5000, 'train')

In [ ]:
viz_latents('exp7', 13000, 'train')

# Experiment-4: 

In [ ]:
df_train = pd.read_csv('./logs/DELPO_omniglot_5-way_5-shot_15-queries/exp4/train.csv')
df_valid = pd.read_csv('./logs/DELPO_omniglot_5-way_5-shot_15-queries/exp4/valid.csv')
df_test = pd.read_csv('./logs/DELPO_omniglot_5-way_5-shot_15-queries/exp4/test.csv')

In [ ]:
df_train

In [ ]:
df_train['accuracy'][-10:-1].mean()

In [ ]:
acc = 100 * np.array(df_test.accuracy)
1.96 * acc.std()/(600**0.5)

In [ ]:
df = plot_summarize([df_train, df_valid, df_test], ['accuracy', 'ELBO', 'Label_KL', 'Style_KL', 'Reconst_Loss', 'CE_Loss'], 20)

In [ ]:
viz_images(['exp4', 'omniglot'], 5000, 'train')

In [ ]:
viz_latents('exp7', 13000, 'train')